# Fast Infrastructure Assessment
- Author: Aidan Marlin
- Date: 20220607
- Version: 1.1
- Description: Jupyter Notebook to automate the execution of key OSINT processes

In [ ]:
target = "www.bing.com"
top_target = "bing.com"
ffuf_subdomain_target = "FUZZ.bing.com"
port = 443

### No need to modify from here
import socket
import os

os.environ["PATH"] = os.environ["PATH"] + ":/run/current-system/sw/bin"

target_ip = socket.gethostbyname(target) 
ffuf_subdomain_target_https = "https://" + ffuf_subdomain_target
ffuf_subdomain_target_http = "http://" + ffuf_subdomain_target
ffuf_directory_brute = target + ":" + str(port) + "/FUZZ"
http_target = "http://" + target
https_target = "https://" + target

!mkdir -p tools/amass
!mkdir -p tools/ctfr
!mkdir -p tools/ffuf_subdomain
!mkdir -p tools/ffuf_directory
!mkdir -p tools/masscan
!mkdir -p tools/photon
!mkdir -p tools/puppeteer
!mkdir -p tools/subfinder
!mkdir -p tools/testssl
!mkdir -p tools/theharvester

def read_file(location):
    with open(location) as f:
        text = f.read()
        print(text)


# Target

In [ ]:
print(target)
print(target_ip)


## Whois

In [ ]:
!dig {target}


# Pentester Details

## IP Address

In [ ]:
!curl https://ifconfig.me/


## Start Time

In [ ]:
!date


## Interface Info

In [ ]:
!ip a


# Web

## Screenshot

In [ ]:
import shutil
from pathlib import Path
home = str(Path.home())

os.system("docker run -u root --shm-size 1G --rm -v " + os.getcwd() + "/tools/puppeteer:/screenshots alekzonder/puppeteer:latest screenshot " + https_target + ":" + str(port) + " 1024x768")
if not os.path.isfile("tools/puppeteer/screenshot_1024_768.png"):
    shutil.copyfile(home + "/git/nixos/resources/404.png", "tools/puppeteer/screenshot_1024_768.png")


![Target](tools/puppeteer/screenshot_1024_768.png)


## WhatWeb

In [ ]:
!docker run --rm guidelacour/whatweb ./whatweb {https_target}:{port}


# Discovery

## httpx

In [ ]:
!echo {target} | httpx -silent -status-code -title -tech-detect -ip -asn


## Subfinder

In [ ]:
!subfinder -d {top_target} -silent -active -oI -oJ -o tools/subfinder/results.txt 2>&1 >/dev/null


In [ ]:
!cat tools/subfinder/results.txt | jq -r '"Domain:  \(.host)\nIP addr: \(.ip)\n"'


## amass

In [ ]:
!amass enum -silent -nocolor -active -json tools/amass/results.txt -d {top_target} -p {port}


In [ ]:
!cat tools/amass/results.txt | jq -r '"##### \(.name)\nIP addr: \(.addresses|.[].ip)\nCIDR: \(.addresses|.[].cidr)\nASN: \(.addresses|.[].asn)\nDesc: \(.addresses|.[].desc)\n"'


## theHarvester

In [ ]:
os.system("docker run --rm -v \"" + os.getcwd() + "/tools/theharvester:/mnt\" simonthomas/theharvester:latest theharvester -d " + top_target + " -b all -f /mnt/results.html")


## ctfr

In [ ]:
os.system("docker run --rm -v \"" + os.getcwd() + "/tools/ctfr/:/mnt\" unapibageek/ctfr -d " + top_target + " -o /mnt/results.txt")


## photon

In [ ]:
!photon --keys -u {https_target} -l 3 -t 100 --wayback -o tools/photon/{target}


### external.txt

In [ ]:
read_file("tools/photon/" + target + "/external.txt")


### files.txt

In [ ]:
read_file("tools/photon/" + target + "/files.txt")


### fuzzable.txt

In [ ]:
read_file("tools/photon/" + target + "/fuzzable.txt")


### intel.txt

In [ ]:
read_file("tools/photon/" + target + "/intel.txt")


### internal.txt

In [ ]:
!cat tools/photon/{target}/internal.txt | httpx -silent -status-code -location -title -tech-detect -cl -ct -fc 401,403,404


### scripts.txt

In [ ]:
read_file("tools/photon/" + target + "/scripts.txt")


## ffuf (subdomains)

In [ ]:
!echo https://{ffuf_subdomain_target}
!ffuf -s -of all -w /home/user/git/wordlists/SecLists/Discovery/DNS/subdomains-top1million-5000.txt -u https://{ffuf_subdomain_target} -od tools/ffuf_subdomain
!echo "`find tools/ffuf_subdomain -type f | wc -l` responses worth investigating.."


## ffuf (directory brute)


In [ ]:
!echo https://{ffuf_directory_brute}
!ffuf -s -ac -sf -of all -w /home/user/git/wordlists/SecLists/Discovery/Web-Content/raft-large-directories.txt -u https://{ffuf_directory_brute} -od tools/ffuf_directory
!echo "`find tools/ffuf_directory -type f | wc -l` responses worth investigating.."
!find tools/ffuf_directory -type f -exec grep -hE 'GET |^HTTP/' {} \; -exec echo \;


## hakrawler

In [ ]:
!echo {https_target} | hakrawler | sort | uniq


# Scanning

## TLS Scan

In [ ]:
!testssl.sh -oA tools/testssl {target}:{port}


## Port Scan (TCP)

In [ ]:
!sudo masscan -p1-65535 {target_ip} --rate=1000 -oG tools/masscan/tcp_results.txt


In [ ]:
read_file("tools/masscan/tcp_results.txt")


## Port Scan (UDP)

In [ ]:
!sudo masscan -pU:1-65535 {target_ip} --rate=1000 -oG tools/masscan/udp_results.txt


In [ ]:
read_file("tools/masscan/udp_results.txt")


# Email Enumeration

In [ ]:
os.system("docker run --rm infoga --domain " + top_target + " --source all --breach -v 2")


# End Time

In [ ]:
!date
